In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt

from scipy.optimize import fmin_tnc
from numpy.linalg import inv
import random

In [ ]:
#CB-MNL
class cb_mnl2:
    def __init__(self, N, K, d, lbd, env_theta):
        super(cb_mnl2, self).__init__()
        self.N=N
        self.K=K
        self.d=d
        self.env_theta = env_theta
        self.X=np.zeros((K,d))[np.newaxis, ...]
        self.Y=np.zeros(K+1)[np.newaxis, ...]
        self.theta=np.ones(d)/(np.linalg.norm(np.ones(d)))
        #self.V=np.zeros((d, d))
        self.mnl = RegularizedMNLRegression()
        self.lbd = lbd
        #self.prev_S = np.zeros(K)
        self.S = np.zeros(K)
        self.max_assort_theta = np.ones(d)/(np.linalg.norm(np.ones(d)))
        
    def choose_S(self,t,x):  # x is N*d matrix
        # generate theta in lattice
        if self.d == 1:
          theta_base_set = np.mgrid[-1:1:.2]
        elif self.d == 2:
          theta_base_set = np.mgrid[-1:1:.2,-1:1:.2,-1:1:.2]
        elif self.d == 3:
          theta_base_set = np.mgrid[-1:1:.2,-1:1:.2,-1:1:.2]
        elif self.d == 4:
          theta_base_set = np.mgrid[-1:1:.2,-1:1:.2,-1:1:.2,-1:1:.2]
        elif self.d == 5:
          theta_base_set = np.mgrid[-1:1:.2,-1:1:.2,-1:1:.2,-1:1:.2,-1:1:.2]

        total_theta = round((theta_base_set.size)/self.d)
        ran = round((total_theta)**(1/self.d))

        beta_sq = .0001*self.d*np.log(self.K*t)
        max_assort_val= 0
        max_assort_single = []
        max_assort_theta =[]
        counter_theta = counter_unitball=0
        for i in range(0,ran):
          for j in range(0,ran):
            for k in range(0,ran):
              theta_val = theta_base_set[:,i,j,k]
              if np.linalg.norm(theta_val) <= 1:
                counter_unitball = counter_unitball+1
                if self.in_confidence_set(theta_val,beta_sq)==1:
                  counter_theta = counter_theta +1
                  means = np.dot(x,theta_val)
                  assort_single = np.argsort(means)[::-1][:self.K]
                  assort_means = np.dot(x[assort_single,:],theta_val)
                  assort_val = self.assort_value(assort_means)
                  if assort_val > max_assort_val:
                    max_assort_single = assort_single
                    max_assort_val = assort_val
                    self.max_assort_theta = theta_val
        
        if len(max_assort_single) ==0:
          means = np.dot(x,theta_val)
          assort_single = np.argsort(means)[::-1][:self.K]
          self.S = assort_single
          print("time", t) 
        else:
          self.S =max_assort_single
        self.X = np.concatenate((self.X, x[self.S,:][np.newaxis, ...]))

        return(self.S)
        

    def update_theta(self,Y,t):
        self.Y = np.concatenate((self.Y, Y[np.newaxis, ...]))
        if t==2:
            self.X = np.delete(self.X, (0), axis=0)
            self.Y = np.delete(self.Y, (0), axis=0)
        self.lbd = 2*d*np.log(self.K*t)
        self.mnl.fit(self.X, self.Y, self.theta, self.lbd)
        self.theta = self.mnl.w
    
    
    def in_confidence_set(self,theta_val,beta_sq):
        
        log_loss_theta_val = self.cost_function(theta_val,self.X,self.Y,lbd)
        log_loss_theta_est = self.cost_function(self.theta,self.X,self.Y,lbd)
        if  log_loss_theta_est < beta_sq + log_loss_theta_val:
          return 1
        else:
          return 0

    def compute_prob(self, theta, x):
        means = np.dot(x, theta)
        u = np.exp(means)
        u_ones = np.column_stack((u,np.ones(u.shape[0])))
        logSumExp = u_ones.sum(axis=1)
        prob = u_ones/logSumExp[:,None]
        return prob

    def cost_function(self, theta, x, y, lam):
        m = x.shape[0]
        prob = self.compute_prob(theta, x)
        return np.sum( np.multiply(y, np.log(prob))) - .5*lam*np.linalg.norm(theta)

    def assort_value(self,means):
        u = np.exp(means)
        uSum = 1 + u.sum()
        rwd = u.sum()/uSum
        return rwd


In [ ]:
#CB-MNL
class cb_mnl_d5:
    def __init__(self, N, K, d, lbd, env_theta):
        super(cb_mnl_d5, self).__init__()
        self.N=N
        self.K=K
        self.d=d
        self.env_theta = env_theta
        self.X=np.zeros((K,d))[np.newaxis, ...]
        self.Y=np.zeros(K+1)[np.newaxis, ...]
        self.theta=np.ones(d)/(np.linalg.norm(np.ones(d)))
        self.mnl = RegularizedMNLRegression()
        self.lbd = lbd
        self.S = np.zeros(K)
        self.max_assort_theta = np.ones(d)/(np.linalg.norm(np.ones(d)))

    
        
    def choose_S(self,t,x):  # x is N*d matrix
        # generate theta in lattice
        if self.d == 1:
          theta_base_set = np.mgrid[-1:1:.2]
        elif self.d == 2:
          theta_base_set = np.mgrid[-1:1:.2,-1:1:.2,-1:1:.2]
        elif self.d == 3:
          theta_base_set = np.mgrid[-1:1:.2,-1:1:.2,-1:1:.2]
        elif self.d == 4:
          theta_base_set = np.mgrid[-1:1:.2,-1:1:.2,-1:1:.2,-1:1:.2]
        elif self.d == 5:
          theta_base_set = np.mgrid[-1:1:.2,-1:1:.2,-1:1:.2,-1:1:.2,-1:1:.2]

        total_theta = round((theta_base_set.size)/self.d)
        ran = round((total_theta)**(1/self.d))

        beta_sq = .0001*self.d*np.log(self.K*t)
        max_assort_val= 0
        max_assort_single = []
        max_assort_theta =[]
        counter_theta = counter_unitball=0
        for i in range(0,ran):
          for j in range(0,ran):
            for k in range(0,ran):
              for l in range(0,ran):
                for m in range(0,ran):
                  theta_val = theta_base_set[:,i,j,k,l,m]
                  if np.linalg.norm(theta_val) <= 1:
                    counter_unitball = counter_unitball+1
                    ### test in confidence set
                    if self.in_confidence_set(theta_val,beta_sq)==1:
                      counter_theta = counter_theta +1
                      means = np.dot(x,theta_val)
                      assort_single = np.argsort(means)[::-1][:self.K]
                      assort_means = np.dot(x[assort_single,:],theta_val)
                      assort_val = self.assort_value(assort_means)
                      if assort_val > max_assort_val:
                        max_assort_single = assort_single
                        max_assort_val = assort_val
                        self.max_assort_theta = theta_val
        
        if len(max_assort_single) ==0:
          means = np.dot(x,theta_val)
          assort_single = np.argsort(means)[::-1][:self.K]
          self.S = assort_single
        else:
          self.S =max_assort_single
        self.X = np.concatenate((self.X, x[self.S,:][np.newaxis, ...]))

        return(self.S)
        

    def update_theta(self,Y,t):
        self.Y = np.concatenate((self.Y, Y[np.newaxis, ...]))
        if t==2:
            self.X = np.delete(self.X, (0), axis=0)
            self.Y = np.delete(self.Y, (0), axis=0)
        self.lbd = 2*d*np.log(self.K*t)
        self.mnl.fit(self.X, self.Y, self.theta, self.lbd)
        self.theta = self.mnl.w
    
    
    def in_confidence_set(self,theta_val,beta_sq):
        
        log_loss_theta_val = self.cost_function(theta_val,self.X,self.Y,lbd)
        log_loss_theta_est = self.cost_function(self.theta,self.X,self.Y,lbd)
        if  log_loss_theta_est < beta_sq + log_loss_theta_val:
          return 1
        else:
          return 0

    def compute_prob(self, theta, x):
        means = np.dot(x, theta)
        u = np.exp(means)
        u_ones = np.column_stack((u,np.ones(u.shape[0])))
        logSumExp = u_ones.sum(axis=1)
        prob = u_ones/logSumExp[:,None]
        return prob

    def cost_function(self, theta, x, y, lam):
        m = x.shape[0]
        prob = self.compute_prob(theta, x)
        return np.sum( np.multiply(y, np.log(prob))) - .5*lam*np.linalg.norm(theta)

    def assort_value(self,means):
        u = np.exp(means)
        uSum = 1 + u.sum()
        rwd = u.sum()/uSum
        return rwd
